In [3]:
import os
import requests

# Remove the file if it exists
file_path = 'minsearch.py'
if os.path.exists(file_path):
    os.remove(file_path)

# Download the new file from internet
url = 'https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py'
response = requests.get(url)
if response.status_code == 200:
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded and saved as {file_path}")
else:
    print("Failed to download the file")


File downloaded and saved as minsearch.py


In [5]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [10]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
from openai import OpenAI
# to know the port number type "ollama start" in console
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [17]:
llm('write this is a test')

' To create an automated script to perform a basic system check and generate a report, I\'ll be using Python along with some of its libraries like `os`, `psutil` for monitoring the system processes, and reporting via simple text output. Below you will find a very simplistic example:\n\n```python\nimport os\nimport sys\nimport psutil\n\n# Define functions to check various aspects of your computer\'s health & status\ndef get_cpu_load():\n    return psutil.getloadavg()[0]  # Get average load for the last 1 minute, excluding idle process(es) and system processes (soil not reported).\n\ndef is_memory_overheating(max_percentage=85):\n    mem = psutil.virtual_memory()\n    return round((mem.used/mem.total)*100, 2) > max_percentage   # Check if memory usage exceeds a threshold (%) and returns True or False accordingly for overheating indication\n    \ndef is_disk_space_low(threshold=35):  # Threshhold to check low disk space situation in percent. e.g., If less than the free hard drive, it mean